In [1]:
from fmm import H3MM,hexs2wkt

### Match to hexagon

In [2]:
from fmm import H3MM,hexs2wkt
from shapely.geometry import LineString,shape, mapping
import json
from ipyleaflet import GeoJSON
from shapely.wkt import dumps, loads
def match_geojson_hex(traj, hex_level, interpolate):
    traj_wkt = shape(traj["geometry"]).wkt
    result = H3MM.match_wkt(traj_wkt, hex_level, interpolate)
    hex_wkt = hexs2wkt(result.hexs)
    return GeoJSON(name="Hex",
                      data=mapping(loads(hex_wkt)), 
                       style = {
                           'fillColor':'green',
                           'opacity':1.0,
                           'fillOpacity': 0.8,
                           'color': 'white'
    })
def traj_geojson_layer(traj):
    return GeoJSON(name="Traj",
                      data=traj, 
                       style = {
                           'fillColor':'orange',
                           'opacity':1,
                           'weight':10
    })

In [3]:
from ipyleaflet import Map, basemaps, WidgetControl, DrawControl
from ipywidgets import IntSlider, ColorPicker, jslink, Checkbox, Button, Layout

m = Map(center=(59.33258,18.0649), zoom=12, 
        basemap=basemaps.OpenStreetMap.Mapnik)

# zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=14)
# jslink((zoom_slider, 'value'), (m, 'zoom'))
# widget_control1 = WidgetControl(widget=zoom_slider, position='topright')
# m.add_control(widget_control1)

h_slider = IntSlider(description='H:', min=7, max=9, value=7)
h_widget_control = WidgetControl(widget=h_slider, position='topright')
m.add_control(h_widget_control)

interpolate_box = Checkbox(
    value=True,
    description='Interpolate',
    disabled=False,
    indent=True
)

interpolate_widget_control = WidgetControl(widget=interpolate_box, position='topright')
m.add_control(interpolate_widget_control)

draw_control = DrawControl(circlemarker={}, polygon={}, edit=False,
    remove=False)
draw_control.polyline =  {
    "shapeOptions": {
        "color": "#fca45d",
        "weight": 6,
        "opacity": 1
    }
}

m.add_control(draw_control)

# Define event listeners

clear_map_btn = Button(
    description='',
    icon = 'fa-trash',
    layout = Layout(width = '28px',padding='0px')
)

clear_map_btn_control = WidgetControl(widget=clear_map_btn, position='topleft',max_width=32)
m.add_control(clear_map_btn_control)

hex_layer = None
traj_layer = None

def clear_map():
    global traj_layer, hex_layer
    if (hex_layer!=None):
        m.remove_layer(hex_layer) 
        hex_layer = None
    if (traj_layer!=None):
        m.remove_layer(traj_layer) 
        traj_layer = None

def update_map(data):
    global traj_layer, hex_layer
    if data==None:
        if traj_layer!=None:
            m.remove_layer(traj_layer)
            if (hex_layer!=None):
                m.remove_layer(hex_layer) 
                hex_layer = None
            temp_hex_layer = match_geojson_hex(traj_layer.data,
                h_slider.value,interpolate_box.value)
            m.add_layer(temp_hex_layer)
            hex_layer = temp_hex_layer
            m.add_layer(traj_layer)
    else:
        temp_hex_layer = match_geojson_hex(data,h_slider.value,interpolate_box.value)
        temp_traj_layer = traj_geojson_layer(data)
        if (hex_layer!=None):
            m.remove_layer(hex_layer) 
            hex_layer = None
        if (traj_layer!=None):
            m.remove_layer(traj_layer)     
            traj_layer = None
        m.add_layer(temp_hex_layer)
        m.add_layer(temp_traj_layer)
        hex_layer = temp_hex_layer
        traj_layer = temp_traj_layer

def on_button_clicked(b):
    clear_map()
    
clear_map_btn.on_click(on_button_clicked)

def on_value_change(change):
    update_map(None)

h_slider.observe(on_value_change, names='value')
interpolate_box.observe(on_value_change, names='value')

def handle_draw(target, action, geo_json):
    if (action=="deleted"):
        m.remove_layer(hex_layer)
        m.remove_layer(traj_layer)
        hex_layer = None
        traj_layer = None
    if (action=="created"):
        draw_control.clear()
        update_map(geo_json)

draw_control.on_draw(handle_draw)

m.layout.height = '540px'
m

TWFwKGNlbnRlcj1bNTkuMzMyNTgsIDE4LjA2NDldLCBjb250cm9scz0oWm9vbUNvbnRyb2wob3B0aW9ucz1bdSdwb3NpdGlvbicsIHUnem9vbV9pbl90ZXh0JywgdSd6b29tX2luX3RpdGxlJyzigKY=


### Match a GeoJSON object

In [6]:
example_geojson = {u'geometry': {u'type': u'LineString', u'coordinates': [[17.995665, 59.323298], [18.057295, 59.320944], [18.075285, 59.354437], [18.125415, 59.326594]]}, u'type': u'Feature', u'properties': {u'style': {u'opacity': 1, u'weight': 6, u'color': u'#fca45d', u'stroke': True, u'clickable': True, u'fill': False}}}

In [7]:
clear_map()
update_map(example_geojson)